In [1]:
#Imports
import glob
import os
import pandas as pd

In [16]:
#Adding a month and a day to each year in epsi
pre_path = "C:\\Users\\millen11\\Dropbox\\PC\\Documents\\academia\\rpi\\classes\\fall23\\introToML\\introToMLapps\\project\\dataset\\cleaned\\"
epsi_df = pd.read_csv(pre_path + "epsi_df.csv")

epsi_df["Date"] = epsi_df["Year"].apply(lambda year: str(year) + "-01-31")
epsi_df = epsi_df.drop(columns=["Unnamed: 0"])
display(epsi_df)

epsi_df.to_csv("epsi_df.csv", index = False)


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\millen11\\Dropbox\\PC\\Documents\\academia\\rpi\\classes\\fall23\\introToML\\introToMLapps\\project\\dataset\\cleaned\\epsi_df.csv'

In [37]:
#Read in external facing indices
ecpi_df = pd.read_csv(pre_path + "ecpi_df.csv")
ccpi_df = pd.read_csv(pre_path + "ccpi_df.csv")
epsi_df = pd.read_csv(pre_path + "epsi_df.csv")
gscpi_df = pd.read_csv(pre_path + "gscpi_df.csv")

#Merge ecpi and ccpi
ecpi_ccpi_df = pd.merge(ecpi_df, ccpi_df, on = ["Date","Country"], how = "inner")
epsi_ecpi_ccpi_df = pd.merge(epsi_df, ecpi_ccpi_df, on = ["Date","Country"], how = "outer")

epsi_ecpi_ccpi_df["EPSI"] = epsi_ecpi_ccpi_df['EPSI'].interpolate(method='linear')
epsi_ecpi_ccpi_df["CCPI"] = epsi_ecpi_ccpi_df['CCPI'].interpolate(method='linear')
epsi_ecpi_ccpi_df["ECPI"] = epsi_ecpi_ccpi_df['ECPI'].interpolate(method='linear')


environmental_indices_df = pd.merge(epsi_ecpi_ccpi_df, gscpi_df, on = ["Date"], how = "outer")
environmental_indices_df["GSCPI"] = environmental_indices_df["GSCPI"].interpolate(method="linear")
display(environmental_indices_df)

,Country,EPSI,Date,ECPI,CCPI,GSCPI
0,Greece,1.000000,1990-01-31,NaN,NaN,NaN
1,Greece,0.000000,1990-01-31,NaN,NaN,NaN
2,Greece,0.000000,1990-01-31,NaN,NaN,NaN
3,Greece,0.361111,1990-01-31,NaN,NaN,NaN
4,Greece,0.000000,1990-01-31,NaN,NaN,NaN
...,...,...,...,...,...,...
31390,NaN,NaN,2023-05-31,NaN,NaN,-1.57
31391,NaN,NaN,2023-06-30,NaN,NaN,-1.11
31392,NaN,NaN,2023-07-31,NaN,NaN,-0.86
31393,NaN,NaN,2023-08-31,NaN,NaN,-1.08


In [ ]:
#Initializing variables and Data
path_to_indices = "C:\\Users\\millen11\\Dropbox\\PC\\Documents\\academia\\rpi\\classes\\fall23\\introToML\\introToMLapps\\project\\dataset\\cleaned\\market_indices"
file_pattern = "*.csv"
file_paths = glob.glob(os.path.join(path_to_indices, file_pattern))

for file_path in file_paths:
  with open(file_path, "r") as file:
    index_df = pd.read_csv(file_path) #Current market index i.e. S&P500
    